# 미니스탑 매장 정보 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import time
import pandas as pd

In [2]:
def ministop_store(location, result):
    wd = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    wd.get("https://www.ministop.co.kr/MiniStopHomePage/page/store/store.do")
    time.sleep(1)
    try:
        wd.execute_script("document.querySelector('#section > div.inner.wrap > div > div.store_content > div.store_txt > ul > li.on > a').click()")

        select = Select(wd.find_element(By.ID, "area1"))
        select.select_by_visible_text(location) # 지역선택

        wd.execute_script("document.querySelector('#area1 > option:nth-child(2)')")
        wd.execute_script('document.querySelector("#section > div.inner.wrap > div > div.store_content > div.store_txt > div.area > a").click()')

        time.sleep(1)
        html = wd.page_source   # 수행된 페이지의 소스코드 저장
        soup = BeautifulSoup(html, 'html.parser') #html 소스 파싱

        store_li = soup.select("div.area > ul > li") # 매장 정보를 갖고 있는 태그 가져오기

        store_list=[] #추출한 태그에서 매장정보를 문자열로 리스트에 저장
        for i in range(0, len(store_li)):
            store_list.append(store_li[i].text)

        # 추출된 문자열 리스트에는 앞, 뒤 공백이 있으므로 제거해줌.
        for s in store_list:
            s = s.strip()   # 앞 뒤 공백제거
            s = s.split("\n\t")
            store_name = s[0]
            store_address = s[1]
            store_tel = s[2]
            result.append([store_name] + [store_address] + [store_tel])
            
    except:
        return None
        
    return 


In [3]:
def main():
    location = input("알고 싶은 미니스톱 매장 지역을 입력 (서울, 경기, 강원, 충남, 충북, 인천, 부산, 광주, 전남, 전북, 경북, 경남, 대구, 울산, 세종, 대전) : ")
    result = []
    ministop_store(location, result) 
    
    
    # csv 파일에 저장
    CB_tbl = pd.DataFrame(result, columns=('store', 'address', 'phone'))
    CB_tbl.to_csv('./data/ministop_store_%s.csv' %(location), encoding='cp949', mode='w', index=True)
    
    print("저장 완료")

In [4]:
if __name__ == '__main__' :
   main()
    

알고 싶은 미니스톱 매장 지역을 입력 (서울, 경기, 강원, 충남, 충북, 인천, 부산, 광주, 전남, 전북, 경북, 경남, 대구, 울산, 세종, 대전) : 서울
저장 완료
